Shyju Kozhisseri<br/>ID: 309572<br/>Group: J41323c

## Import Data

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings("ignore",category=matplotlib.MatplotlibDeprecationWarning)

In [ ]:
input = pd.read_csv('garments_worker_productivity.csv')

In [ ]:
input.head()

In [ ]:
input.describe()

In [ ]:
input.isna().sum()

In [ ]:
sub_sample = input[['date', 'team', 'smv', 'no_of_workers', 'wip', 'over_time', 'incentive', 'idle_time', 'idle_men', 'no_of_style_change', 'targeted_productivity', 'actual_productivity']]
sub_sample

In [ ]:
sub_sample = sub_sample.fillna(sub_sample.mean())
sub_sample.head()

In [ ]:
sample = sub_sample.groupby(['team', 'date']).agg({'smv':'sum', 'no_of_workers':'sum', 'wip':'sum', 'over_time':'sum', 'incentive':'sum', 'idle_time':'sum', 'idle_men':'sum', 'no_of_style_change':'sum', 'targeted_productivity':'max','actual_productivity':'mean'})
sample

## Scale Data

In [ ]:
sample.columns

In [ ]:
scaler = MinMaxScaler()
original_data = sample.copy()
sample[['smv', 'no_of_workers', 'wip', 'over_time', 'incentive', 'idle_time','idle_men', 'no_of_style_change']] = scaler.fit_transform(sample[['smv', 'no_of_workers', 'wip', 'over_time', 'incentive', 'idle_time','idle_men', 'no_of_style_change']])
sample

In [ ]:
sample.describe()

## Stationarity Analysis

In [ ]:
data = sample.reset_index()
data_1 = data.loc[data['team']==1].set_index('date', drop=True)[['actual_productivity']]
data_1.index = pd.to_datetime(data_1.index)
data_1.sort_index(inplace=True)

In [ ]:
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(10, 7))
plt.plot(data_1)
plt.title('Team 1 Productivity', fontsize=14)
plt.xlabel('Date', fontsize=12)
plt.xticks(rotation = 45)
plt.ylabel('Actual Productivity', fontsize=12)
plt.show()

### Moving Mean and Variance Test

In [ ]:
split = len(data_1)//2
x1, x2 = data_1[:split], data_1[split:]
print("Mean of the splits are: ", x1.mean()['actual_productivity'], x2.mean()['actual_productivity'])
print("Variance of the splits are: ", x1.var()['actual_productivity'], x2.var()['actual_productivity'])

### Augmented Dickey-Fuller test

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(data_1)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

# p value<0.05, data does not have a unit root and is stationary

## Covariance and Correlation

In [ ]:
sample.cov()

In [ ]:
sample.corr()

In [ ]:
corrMatrix = sample.corr()
plt.figure(figsize=(15, 8))
sns.heatmap(corrMatrix, annot=True, cmap='Blues')
plt.show()

## Noise Filtering

In [ ]:
data_freq = data_1.asfreq('D', method='ffill')
data_freq_mean = data_freq.resample('W').mean()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(data_freq['actual_productivity'],
marker='.', linestyle='-', linewidth=0.5, label='Daily')
ax.plot(data_freq_mean['actual_productivity'],
marker='o', markersize=8, linestyle='-', label='Weekly Mean')
ax.set_ylabel('Actual Productivity')
ax.legend()

### Savgol Filter

In [ ]:
from scipy.signal import savgol_filter
ts = data_freq['actual_productivity']
x = np.arange(0, 70)


n = 10
a = len(ts.index)-1
ts_filter = savgol_filter(ts, a, n)
tsf = pd.Series(ts_filter)
tsf.index = ts.index

fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(ts, marker='.', linestyle='-', linewidth=0.5, label='Original')
ax.plot(tsf, marker='o', linestyle='-', linewidth=0.5, label='Filtered')
ax.set_ylabel('Actual Productivity')
ax.legend()


### Gaussian Filter

In [ ]:
from scipy.ndimage import filters
from scipy.signal import convolve, gaussian

windows_size=70
gauss_sd=3
window = gaussian(windows_size, gauss_sd)
data_gaussian_filter = dict()

output_gauss=convolve(ts, window/window.sum(), mode='same')
output = pd.Series(output_gauss, name='actual_productivity', index=ts.index)

plt.figure(figsize=(12, 8))
plt.plot(ts, marker='.', linestyle='-', linewidth=0.5, label ='Original')
plt.plot(output, marker='o', linestyle='-', linewidth=0.5, label = 'Filtered')
plt.legend(loc='upper left')
plt.show()

## Spectral Density Function

### Without Filter

In [ ]:
import matplotlib.mlab as mlab

frame_per_second = 1000
fig = plt.figure(figsize=(15, 8))
ax = plt.axes()

sdf = plt.psd(ts, NFFT = 256, Fs = frame_per_second, window = mlab.window_none, scale_by_freq = True)


### With filter

In [ ]:
fig = plt.figure(figsize=(15, 8))
ax = plt.axes()

sdf_2 = plt.psd(tsf, NFFT = 256, Fs = frame_per_second, window = mlab.window_none, scale_by_freq = True)


## Autoregression Model

### On Unfiltered Data

In [ ]:
pd.plotting.lag_plot(ts)

In [ ]:
pd.plotting.autocorrelation_plot(ts)

In [ ]:
values = pd.DataFrame(ts.values, dtype=np.float64)
dataframe = pd.concat([values.shift(1), values], axis=1)
dataframe.columns = ['t', 't+1']
X = dataframe.values
train_size = 50
train, test = X[1:train_size], X[train_size:]
train_X, train_y = train[:,0], train[:,1]
test_X, test_y = test[:,0], test[:,1]

# persistence model on training set
train_pred = [x for x in train_X]

# residuals
train_resid = [train_y[i]-train_pred[i] for i in range(len(train_pred))]

# model the training set residuals
window = 7
model = AutoReg(train_resid, lags=window, old_names=True)
model_fit = model.fit()
coef = model_fit.params

history = train_resid[len(train_resid)-window:]
history = [history[i] for i in range(len(history))]
predictions = list()
expected_error = list()
for t in range(len(test_y)):

	# persistence
	yhat = test_X[t]
	error = test_y[t] - yhat
	expected_error.append(error)

	# predict error
	length = len(history)
	lag = [history[i] for i in range(length-window,length)]
	pred_error = coef[0]
	for d in range(window):
		pred_error += coef[d+1] * lag[window-d-1]

    # correct the prediction
	yhat = yhat + pred_error
	predictions.append(yhat)
	history.append(error)
	print('predicted=%f, expected=%f' % (yhat, test_y[t]))
# Metrics
print("\nR2 Score: ", r2_score(test_y, predictions))
print("RMSE: ", np.sqrt(mean_squared_error(test_y, predictions)))
print("Mean Squared Error: ", mean_squared_error(test_y, predictions))
print("Mean Absolute Error: ", mean_absolute_error(test_y, predictions))
residual_uf = test_y - predictions

In [ ]:
# plot predicted error
plt.figure(figsize=(10, 6))
dt = ts[len(ts)-20:].index.values
plt.plot(dt, test_y, label = 'Actual Productivity')
plt.plot(dt, predictions, color='red', label='Predicted Productivity')
plt.xticks(rotation=45)
plt.legend()
plt.show()

### On filtered Data

In [ ]:
pd.plotting.lag_plot(tsf)

In [ ]:
pd.plotting.autocorrelation_plot(tsf)

In [ ]:
values = pd.DataFrame(tsf.values, dtype=np.float64)
dataframe = pd.concat([values.shift(1), values], axis=1)
dataframe.columns = ['t', 't+1']
X = dataframe.values
train_size = 50
train, test = X[1:train_size], X[train_size:]
train_X, train_y = train[:,0], train[:,1]
test_X, test_y = test[:,0], test[:,1]

# persistence model on training set
train_pred = [x for x in train_X]

# residuals
train_resid = [train_y[i]-train_pred[i] for i in range(len(train_pred))]

# model the training set residuals
window = 15
model = AutoReg(train_resid, lags=window, old_names=True)
model_fit = model.fit()
coef = model_fit.params

history = train_resid[len(train_resid)-window:]
history = [history[i] for i in range(len(history))]
predictions = list()
expected_error = list()
for t in range(len(test_y)):

	# persistence
	yhat = test_X[t]
	error = test_y[t] - yhat
	expected_error.append(error)

	# predict error
	length = len(history)
	lag = [history[i] for i in range(length-window,length)]
	pred_error = coef[0]
	for d in range(window):
		pred_error += coef[d+1] * lag[window-d-1]

    # correct the prediction
	yhat = yhat + pred_error
	predictions.append(yhat)
	history.append(error)
	print('predicted=%f, expected=%f' % (yhat, test_y[t]))
# metrics
print("\nR2 Score: ", r2_score(test_y, predictions))
print("RMSE: ", np.sqrt(mean_squared_error(test_y, predictions)))
print("Mean Squared Error: ", mean_squared_error(test_y, predictions))
print("Mean Absolute Error: ", mean_absolute_error(test_y, predictions))
residual_f = test_y - predictions

In [ ]:
plt.figure(figsize=(10, 6))
dt = tsf[len(tsf)-20:].index.values
plt.plot(dt, test_y, label = 'Actual Productivity')
plt.plot(dt, predictions, color='red', label='Predicted Productivity')
plt.xticks(rotation=45)
plt.legend()
plt.show()

## Model as a Linear Dynamical System

In [ ]:
input_data = data.loc[data['team']==1].set_index('date', drop=True)
input_data.index = pd.to_datetime(input_data.index)
input_data.sort_index(inplace=True)
input_data = input_data[['smv', 'wip', 'incentive', 'targeted_productivity', 'actual_productivity']].asfreq('D', method='ffill')
input_data

In [ ]:
from statsmodels.tsa.api import VAR

model = VAR(input_data, freq="1D")
model.select_order(10).summary()

In [ ]:
results = model.fit(7)
print(results.summary())

In [ ]:
def add_lags(s, lag, tar):
    new_dict={}
    for col_name in s:
        new_dict[col_name]=s[col_name]
        if col_name not in tar:
            for l in range(1,lag+1):
                new_dict['%s_lag%d' %(col_name,l)]=s[col_name].shift(l)
    res=pd.DataFrame(new_dict,index=s.index)
    return res

In [ ]:
from sklearn.ensemble import RandomForestRegressor

def fit_and_predict(X, y, n=20):
    lr = RandomForestRegressor(n_estimators = 10)
    lr.fit(X[:len(X)-n], y[:len(y)-n].ravel())
    preds = lr.predict(X[len(X)-n:])
    plt.figure(figsize=(15, 8))
    plt.plot(X[len(X)-n:].index.values, y[len(y)-n:], label='Original Data')
    plt.plot(X[len(X)-n:].index.values, preds, label='Predicted Data')
    plt.xticks(rotation='45')
    plt.legend()
    #plt.savefig('filtered.png')
    plt.show()
    y_true = y[len(y)-n:].reshape(n,)
    print("R2 Score: ", r2_score(y_true, preds))
    print("RMSE: ", np.sqrt(mean_squared_error(y_true, preds)))
    print("Mean Squared Error: ", mean_squared_error(y_true, preds))
    print("Mean Absolute Error: ", mean_absolute_error(y_true, preds))
    return (y_true-preds)


### Target => actual_productivity

In [ ]:
df = add_lags(s=input_data, lag=7, tar=['actual_productivity']).dropna()
X = df.loc[:, df.columns != 'actual_productivity']
y = df[['actual_productivity']].values
print(X.columns)
residual_uflds = fit_and_predict(X, y, 20)

### Target => wip

In [ ]:
df = add_lags(s=input_data, lag=7, tar=['wip']).dropna()
X = df.loc[:, df.columns != 'wip']
y = df[['wip']].values
print(X.columns)
_ = fit_and_predict(X, y, 20)

### Target => smv

In [ ]:
df = add_lags(s=input_data, lag=7, tar=['smv']).dropna()
X = df.loc[:, df.columns != 'smv']
y = df[['smv']].values
print(X.columns)
_ = fit_and_predict(X, y, 20)

### Target => targeted_productivity

In [ ]:
df = add_lags(s=input_data, lag=7, tar=['targeted_productivity']).dropna()
X = df.loc[:, df.columns != 'targeted_productivity']
y = df[['targeted_productivity']].values
print(X.columns)
_ = fit_and_predict(X, y, 20)

### Filtered Data

In [ ]:
input_data['actual_productivity'] = tsf.values
input_data

In [ ]:
df = add_lags(s=input_data, lag=7, tar=['actual_productivity']).dropna()
X = df.loc[:, df.columns != 'actual_productivity']
y = df[['actual_productivity']].values
print(X.columns)
residual_flds = fit_and_predict(X, y, 20)

## Residual Analysis

### AutoReg - Unfiltered

In [ ]:
sns.histplot(residual_uf, kde=True, stat='density', label='residuals', bins=10)
plt.ylabel('density')
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,2.5))
_, (__, ___, r) = stats.probplot(residual_uf, plot=ax, fit=True)

### AutoReg - Filtered

In [ ]:
sns.histplot(residual_f, kde=True, stat='density', label='residuals', bins=10)
plt.ylabel('density')
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,2.5))
_, (__, ___, r) = stats.probplot(residual_f, plot=ax, fit=True)

### LDS Unfiltered

In [ ]:
sns.histplot(residual_uflds, kde=True, stat='density', label='residuals', bins=10)
plt.ylabel('density')
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,2.5))
_, (__, ___, r) = stats.probplot(residual_uflds, plot=ax, fit=True)

### LDS Filtered

In [ ]:
sns.histplot(residual_flds, kde=True, stat='density', label='residuals', bins=10)
plt.ylabel('density')
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6,2.5))
_, (__, ___, r) = stats.probplot(residual_flds, plot=ax, fit=True)